# Activity 4 - INSERT, UPDATE, DELETE

We'll do a few simple INSERT, UPDATE, DELETE queries with an in-memory database. These queries are extremely simple in syntax.

In [1]:
import sqlite3
# Database setup: Please don't change this
db = sqlite3.connect("file::memory:?cache=shared")

db.executescript("""
drop table if exists ProgrammingLanguage;
CREATE TABLE ProgrammingLanguage (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name text not null,
  github_stars integer null,

  CONSTRAINT constraint_name UNIQUE (name)
);
""")

### INSERT

```sql
INSERT INTO <TABLE> (<col1>, <col2>, ...) VALUES (<val1>, <val2>, ...)
```


In [2]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[]

**Simple INSERT**

In [3]:
cursor = db.execute("""
    INSERT INTO ProgrammingLanguage
        (id, name, github_stars)
    VALUES
        (1, "Python", 18856)
""")

In [4]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[(1, 'Python', 18856)]

**Insert without an ID, as it's AUTOINCREMENT**

In [5]:
cursor = db.execute("""
    INSERT INTO ProgrammingLanguage
        (name, github_stars)
    VALUES
        ("Ruby", 14126)
""")

Want to know which ID was assigned? Use `cursor.lastrowid`:

In [6]:
cursor.lastrowid

2

In [7]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[(1, 'Python', 18856), (2, 'Ruby', 14126)]

**Constraint checks happen at INSERT time**

For example, duplicating a Programming Language:

In [8]:
cursor = db.execute("""
    INSERT INTO ProgrammingLanguage
        (name, github_stars)
    VALUES
        ("Ruby", 14126)
""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: UNIQUE constraint failed: ProgrammingLanguage.name

**NULLable fields are optional**

In [9]:
cursor = db.execute("""
    INSERT INTO ProgrammingLanguage
        (name)
    VALUES
        ("Node JS")
""")

In [10]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[(1, 'Python', 18856), (2, 'Ruby', 14126), (3, 'Node JS', None)]

**INSERT multiple values with the same query**

In [11]:
cursor = db.execute("""
    INSERT INTO ProgrammingLanguage
        (name, github_stars)
    VALUES
        ("Rust", 27319),
        ("Perl", 321),
        ("Clojure", 7056),
        ("Scala", 9933),
        ("Elixir", 12610),
        ("Coffee Script", 14636),
        ("PHP", 16864)
""")

In [12]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[(1, 'Python', 18856),
 (2, 'Ruby', 14126),
 (3, 'Node JS', None),
 (4, 'Rust', 27319),
 (5, 'Perl', 321),
 (6, 'Clojure', 7056),
 (7, 'Scala', 9933),
 (8, 'Elixir', 12610),
 (9, 'Coffee Script', 14636),
 (10, 'PHP', 16864)]

**INSERT also works with param replacing**

In [13]:
params = {
    'name': 'Kotlin',
    'stars': 21592
}
cursor = db.execute("""
    INSERT INTO ProgrammingLanguage
        (name, github_stars)
    VALUES
        (:name, :stars)
""", params)

In [14]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[(1, 'Python', 18856),
 (2, 'Ruby', 14126),
 (3, 'Node JS', None),
 (4, 'Rust', 27319),
 (5, 'Perl', 321),
 (6, 'Clojure', 7056),
 (7, 'Scala', 9933),
 (8, 'Elixir', 12610),
 (9, 'Coffee Script', 14636),
 (10, 'PHP', 16864),
 (11, 'Kotlin', 21592)]

### UPDATE

```sql
UPDATE <TABLE>
SET col1 = value1, col22 = value2, ...
[WHERE <CONDITION>];
```

**Conditions are VERY Important**

You want to use a condition 99% of the time.

In [15]:
params = {
    'stars': 46936
}
cursor = db.execute("""
    UPDATE ProgrammingLanguage
    SET
        github_stars = :stars 
    WHERE
    name = "Node JS"
""", params)

In [16]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[(1, 'Python', 18856),
 (2, 'Ruby', 14126),
 (3, 'Node JS', 46936),
 (4, 'Rust', 27319),
 (5, 'Perl', 321),
 (6, 'Clojure', 7056),
 (7, 'Scala', 9933),
 (8, 'Elixir', 12610),
 (9, 'Coffee Script', 14636),
 (10, 'PHP', 16864),
 (11, 'Kotlin', 21592)]

**Update can also self-reference the same table**

In [17]:
cursor = db.execute("""
    UPDATE ProgrammingLanguage
    SET
        github_stars = github_stars * 1000
    WHERE name = "Python"
""", params)

In [18]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[(1, 'Python', 18856000),
 (2, 'Ruby', 14126),
 (3, 'Node JS', 46936),
 (4, 'Rust', 27319),
 (5, 'Perl', 321),
 (6, 'Clojure', 7056),
 (7, 'Scala', 9933),
 (8, 'Elixir', 12610),
 (9, 'Coffee Script', 14636),
 (10, 'PHP', 16864),
 (11, 'Kotlin', 21592)]

**If you DON'T use a condition**

In [19]:
cursor = db.execute("""
    UPDATE ProgrammingLanguage
    SET
        github_stars = 1
""", params)

In [20]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[(1, 'Python', 1),
 (2, 'Ruby', 1),
 (3, 'Node JS', 1),
 (4, 'Rust', 1),
 (5, 'Perl', 1),
 (6, 'Clojure', 1),
 (7, 'Scala', 1),
 (8, 'Elixir', 1),
 (9, 'Coffee Script', 1),
 (10, 'PHP', 1),
 (11, 'Kotlin', 1)]

You've updated your entire table.

### DELETE

```sql
DELETE FROM <TABLE>
WHERE <CONDITION>
```

**CONDITIONS ARE IMPORTANT!**

We're want to delete a programming language from this list (obviously PHP 🤢)...

In [21]:
cursor = db.execute('DELETE FROM ProgrammingLanguage WHERE name = "PHP"')

In [22]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[(1, 'Python', 1),
 (2, 'Ruby', 1),
 (3, 'Node JS', 1),
 (4, 'Rust', 1),
 (5, 'Perl', 1),
 (6, 'Clojure', 1),
 (7, 'Scala', 1),
 (8, 'Elixir', 1),
 (9, 'Coffee Script', 1),
 (11, 'Kotlin', 1)]

PHP is gone! 🎉

**WARNING: If you forget the condition**

In [23]:
cursor = db.execute('DELETE FROM ProgrammingLanguage')

In [24]:
cursor = db.execute('SELECT * FROM ProgrammingLanguage')
cursor.fetchall()

[]

You've just wiped all your data.

**Recommendation:** If you have to write an UPDATE or DELETE query, start writing FIRST the condition. And then complete the table and the rest of your query.